In [1]:
import os
import pandas as pd
import re

# --- 各カテゴリのフォルダを指定 ---
folders = {
    '倒産かつ連結': 'C:/bankdata/tousan/renketsu/',
    '倒産かつ単体': 'C:/bankdata/tousan/tantai/',
    '非倒産かつ連結': 'C:/bankdata/hitousan/renketsu/',
    '非倒産かつ単体': 'C:/bankdata/hitousan/tantai/'
}

# --- カテゴリごとのフラグ設定 ---
category_flags = {
    '倒産かつ連結': {'倒産フラグ': 1, '連結フラグ': 1},
    '倒産かつ単体': {'倒産フラグ': 1, '連結フラグ': 0},
    '非倒産かつ連結': {'倒産フラグ': 0, '連結フラグ': 1},
    '非倒産かつ単体': {'倒産フラグ': 0, '連結フラグ': 0}
}

# --- 全データを格納する辞書 ---
final_data = {}

# --- 各フォルダのCSVファイルを読み込み、横展開 ---
for category, folder_path in folders.items():
    for filename in os.listdir(folder_path):
        if not filename.endswith(".csv"):
            continue
        file_path = os.path.join(folder_path, filename)

        # 証券番号の抽出（先頭4桁 or E+5桁）
        match = re.match(r'(E?\d{4,5})', filename)
        if not match:
            continue
        company_code = match.group(1)

        # 4行目（index=3）をシート名として取得
        with open(file_path, encoding='utf-8') as f:
            lines = f.readlines()
        if len(lines) < 4:
            continue
        sheet_label = lines[3].strip().replace(",", "").replace(" ", "").replace("\n", "")

        # CSV読み込み
        df = pd.read_csv(file_path, encoding='utf-8')

        if df.shape[1] < 2:
            continue  # 列不足スキップ

        # パラメータ列の候補を判定（2列目 or 3列目）
        def numeric_ratio(col_index):
            return pd.to_numeric(df.iloc[:, col_index], errors='coerce').notna().mean()
        
        col2_ratio = numeric_ratio(1)
        col3_ratio = numeric_ratio(2) if df.shape[1] >= 3 else 0
        value_col = 1 if col2_ratio >= col3_ratio else 2

        # ラベル加工（1列目）＋ パラメータ取得
        raw_labels = df.iloc[:, 0].astype(str).str.strip().str.replace(" ", "").str.replace("\n", "")
        labels = [f"{sheet_label}_{label}" for label in raw_labels]
        print(labels)
        values = df.iloc[:, value_col]

        # 辞書初期化
        if company_code not in final_data:
            final_data[company_code] = {'証券番号': company_code}

        # 横方向に追加
        for label, value in zip(labels, values):
            final_data[company_code][label] = value

        # 倒産・連結フラグ追加
        final_data[company_code]['倒産フラグ'] = category_flags[category]['倒産フラグ']
        final_data[company_code]['連結フラグ'] = category_flags[category]['連結フラグ']

# --- DataFrameに変換 ---
final_df = pd.DataFrame.from_dict(final_data, orient='index')

# --- 保存 ---
output_path = 'C:/bankdata/combined_dataset.csv'
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"結合されたデータセットを {output_path} に保存しました。")


結合されたデータセットを C:/bankdata/combined_dataset.csv に保存しました。


In [19]:
import pandas as pd
import numpy as np

# 同じに扱えるラベルのリストをさらに拡張
label_mapping = {
    '経常損失（△）': '経常損益',
    '経常利益又は経常損失（△）': '経常損益',
    '経常利益': '経常損益',
    '当期純損失（△）': '当期純損益',
    '当期純利益又は当期純損失（△）': '当期純損益',
    '当期純利益': '当期純損益',
    '売上総損失（△）': '売上総損益',
    '売上総利益又は売上総損失（△）': '売上総損益',
    '売上総利益': '売上総損益',
    '法人税、住民税及び事業税': '法人税等',
    '法人税等': '法人税等',
    '売上高': '営業収入',
    '営業収入': '営業収入',
    '現金及び預金': '現金預金',
    '現金預金': '現金預金',
    '現金及び現金同等物': '現金預金',
    '棚卸資産': '貯蔵品',
    '貯蔵品': '貯蔵品',
    '原材料及び貯蔵品': '貯蔵品',
    '退職給付引当金': '退職給付に係る負債',
    '退職給付に係る負債': '退職給付に係る負債',

}

# データセット格納用辞書
final_data = {}

# 重複して統合できないラベルを記録するリスト
conflict_log = []

# CSVファイルを読み込み、証券番号ごとのデータを横方向に統合
file_path = 'C:/bankdata/combined_dataset.csv'  # 処理するCSVファイルのパス
df = pd.read_csv(file_path)

# 証券番号の列を特定（ここでは '証券番号' が列名だと仮定）
security_code_column = '証券番号'

# 各証券番号ごとにラベルを統合
for _, row in df.iterrows():
    security_code = row[security_code_column]

    # 証券番号に基づいて辞書を初期化
    if security_code not in final_data:
        final_data[security_code] = {'証券番号': security_code}
    
    # 各ラベルに対して処理を行う
    for label, value in row.items():
        # ラベルがマッピング対象である場合は統一
        unified_label = label_mapping.get(label, label)
        
        # 値が既に存在する場合、`nan`ならそのまま統合
        if unified_label in final_data[security_code]:
            if pd.notna(final_data[security_code][unified_label]):
                if pd.notna(value):
                    conflict_log.append(f"証券番号 {security_code} のラベル {unified_label} に既存の値 {final_data[security_code][unified_label]} と新しい値 {value} が競合しています。")
                continue
        
        # nanでない値を統合
        if pd.notna(value):
            final_data[security_code][unified_label] = value

# 辞書をデータフレームに変換
final_df = pd.DataFrame.from_dict(final_data, orient='index')

# 統合されたデータをCSVファイルとして保存
output_path = 'C:/bankdata/outputfile.csv'  # 保存先のパス
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"ラベルが統合されたデータを {output_path} に保存しました。")

# 競合が発生したラベルのログを出力
if conflict_log:
    with open('C:/bankdata/conflict_log.txt', 'w', encoding='utf-8') as f:
        for log_entry in conflict_log:
            f.write(log_entry + '\n')
    print(f"競合が発生したラベルの詳細は conflict_log.txt に保存されました。")
else:
    print("競合は発生しませんでした。")


ラベルが統合されたデータを C:/bankdata/outputfile.csv に保存しました。
競合が発生したラベルの詳細は conflict_log.txt に保存されました。


In [5]:
import os
import pandas as pd
import re

# 各カテゴリのフォルダを指定
folders = {
    '倒産かつ連結': 'C:/bankdata/tousan/renketsu/',
    '倒産かつ単体': 'C:/bankdata/tousan/tantai/',
    '非倒産かつ連結': 'C:/bankdata/hitousan/renketsu/',
    '非倒産かつ単体': 'C:/bankdata/hitousan/tantai/'
}

# カテゴリごとのフラグ設定
category_flags = {
    '倒産かつ連結': {'倒産フラグ': 1, '連結フラグ': 1},
    '倒産かつ単体': {'倒産フラグ': 1, '連結フラグ': 0},
    '非倒産かつ連結': {'倒産フラグ': 0, '連結フラグ': 1},
    '非倒産かつ単体': {'倒産フラグ': 0, '連結フラグ': 0}
}

# 全データを格納する辞書
final_data = {}

# 各フォルダのCSVファイルを読み込み、企業ごとのデータを横方向に展開
for category, folder_path in folders.items():
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            
            # 証券番号の抽出（先頭4桁、Eが先頭の場合は5桁）
            match = re.match(r'(E?\d{4,5})', filename)
            if match:
                company_code = match.group(1)  # 証券番号
                
                # CSVファイルを読み込む
                df = pd.read_csv(file_path, encoding='utf-8')  # UTF-8エンコーディングで読み込む
                
                # 2つ目の列を分類ラベル、3つ目の列をパラメーターとして使用
                if df.shape[1] >= 3:  # 列数が3以上ある場合
                    labels = df.iloc[:, 0].apply(lambda x: str(x).strip().replace(" ", "").replace("\n", ""))  # 余計なスペースや改行を除去
                    values = df.iloc[:, 2]  # 3つ目の列がパラメーター

                    # 企業ごとのデータが辞書にない場合、新規に作成
                    if company_code not in final_data:
                        final_data[company_code] = {'証券番号': company_code}
                    
                    # ラベルとパラメータを辞書に追加（横方向に展開）
                    for label, value in zip(labels, values):
                        final_data[company_code][label] = value
                    
                    # 倒産フラグと連結フラグを追加
                    final_data[company_code]['倒産フラグ'] = category_flags[category]['倒産フラグ']
                    final_data[company_code]['連結フラグ'] = category_flags[category]['連結フラグ']

# 辞書をデータフレームに変換
final_df = pd.DataFrame.from_dict(final_data, orient='index')

# 結合されたデータをCSVファイルとして保存
output_path = 'C:/bankdata/combined_dataset.csv'
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')  # UTF-8-sigで保存

print(f"結合されたデータセットを {output_path} に保存しました。")

結合されたデータセットを C:/bankdata/combined_dataset.csv に保存しました。


In [20]:
import pandas as pd
from difflib import SequenceMatcher
from collections import defaultdict
import itertools

# --- データ読み込み ---
file_path = 'C:/bankdata/combined_dataset.csv'
df = pd.read_csv(file_path)

# 各ラベルが何社に登場したかをカウント（NaNでない値が存在する行数）
label_usage = df.notna().sum(axis=0).to_dict()

# 使用企業が2社以上のラベルだけ対象にする
usable_columns = [col for col, count in label_usage.items() if count >= 2]

# --- プレフィックスでグルーピング ---
prefix_dict = defaultdict(list)
for col in usable_columns:
    prefix = col[:3]
    prefix_dict[prefix].append(col)

# --- 類似比較（同一プレフィックス内） ---
threshold = 0.8
similar_pairs = []

for group in prefix_dict.values():
    if len(group) < 2:
        continue
    for col1, col2 in itertools.combinations(group, 2):
        ratio = SequenceMatcher(None, col1, col2).ratio()
        if threshold <= ratio < 1.0:
            count1 = label_usage.get(col1, 0)
            count2 = label_usage.get(col2, 0)
            representative = col1 if count1 >= count2 else col2
            similar_pairs.append((col1, col2, round(ratio, 3), count1, count2, representative))

# --- 保存 ---
if similar_pairs:
    result_df = pd.DataFrame(
        similar_pairs,
        columns=['ラベルA', 'ラベルB', '類似度', 'ラベルA使用数', 'ラベルB使用数', '代表ラベル']
    )
    output_path = 'C:/bankdata/similar_labels_filtered.csv'
    result_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"類似ラベル候補を {output_path} に保存しました。")
else:
    print("類似ラベル候補は見つかりませんでした。")


C:\Users\naoyasu.ikeda\AppData\Local\Temp\ipykernel_15488\2750855486.py:8: DtypeWarning: Columns (0,1,8,9,11,13,18,20,21,23,25,27,29,30,37,38,39,40,41,44,45,46,48,51,53,54,55,56,58,59,60,64,66,67,68,72,75,76,78,79,81,82,84,87,88,89,92,93,99,100,101,103,104,113,114,116,118,119,122,123,124,125,126,127,128,129,130,134,135,137,138,139,140,141,142,148,149,150,151,152,153,155,156,157,158,159,160,161,162,164,165,166,167,169,173,175,181,184,185,186,187,188,189,199,202,205,206,214,229,231,270,271,272,274,281,283,286,288,289,290,291,297,298,299,300,302,305,309,311,312,315,316,317,320,325,327,329,330,337,339,340,349,352,353,357,361,362,363,364,365,366,367,368,378,379,382,383,384,385,389,394,395,396,398,399,402,408,409,414,415,417,420,426,428,440,443,444,445,446,448,449,450,452,453,454,456,457,458,459,460,461,462,463,474,480,481,482,483,484,491,493,501,503,506,507,508,511,522,525,529,532,533,536,537,538,539,541,546,547,549,551,552,553,557,560,561,562,564,565,567,574,577,599,602,607,610,620,621,623

類似ラベル候補を C:/bankdata/similar_labels_filtered.csv に保存しました。


In [22]:
import pandas as pd

# 類似ラベルペアとラベル出現頻度を読み込む
similar_df = pd.read_csv('C:/bankdata/similar_labels_filtered.csv')
dataset_df = pd.read_csv('C:/bankdata/combined_dataset.csv', nrows=1)

# 各ラベルの非欠損（非NaN）数を算出
label_usage = dataset_df.notna().sum().to_dict()

# 使用頻度の高い方を代表ラベルとして記録
initial_mapping = {}

for _, row in similar_df.iterrows():
    a, b = row['ラベルA'], row['ラベルB']
    count_a = label_usage.get(a, 0)
    count_b = label_usage.get(b, 0)

    if count_a >= count_b:
        initial_mapping[b] = a  # b → a
    else:
        initial_mapping[a] = b  # a → b

# --- 連鎖解決関数（A→B→C なら A→C） ---
def resolve_chain(mapping):
    resolved = {}

    def find_root(label):
        path = []
        while label in mapping:
            path.append(label)
            label = mapping[label]
            if label in path:  # 循環参照の防止
                break
        return label

    for key in mapping:
        root = find_root(key)
        resolved[key] = root

    return resolved

# --- 連鎖統合を解決 ---
final_mapping = resolve_chain(initial_mapping)

# --- 保存 ---
mapping_df = pd.DataFrame(list(final_mapping.items()), columns=['旧ラベル', '統一ラベル'])
output_path = 'C:/bankdata/label_mapping.csv'
mapping_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"連鎖統合を含むラベルマッピングを {output_path} に保存しました。")


連鎖統合を含むラベルマッピングを C:/bankdata/label_mapping.csv に保存しました。


In [25]:
import pandas as pd
import numpy as np

# データとマッピングを読み込み
df = pd.read_csv('C:/bankdata/combined_dataset.csv')
mapping_df = pd.read_csv('C:/bankdata/label_mapping.csv')

# マッピング辞書
mapping_dict = dict(zip(mapping_df['旧ラベル'], mapping_df['統一ラベル']))

# 競合チェック結果格納用
results = []

for old_label, new_label in mapping_dict.items():
    if old_label in df.columns:
        if new_label in df.columns:
            # 両方の列に値がある行を抽出
            conflict_mask = (df[old_label].notna()) & (df[new_label].notna())
            conflict_rows = df.loc[conflict_mask].copy()
            conflict_codes = conflict_rows['証券番号'].astype(str).tolist()
            conflict_count = len(conflict_codes)

            # 厳密な数値の競合があるかどうかを判定
            strong_conflict = False
            for _, row in conflict_rows.iterrows():
                old_val = row[old_label]
                new_val = row[new_label]

                # 片方が数値でない場合は競合としない
                try:
                    old_num = float(old_val)
                    new_num = float(new_val)
                    if not np.isclose(old_num, new_num, rtol=1e-5, equal_nan=True):
                        strong_conflict = True
                        break
                except:
                    continue  # 数値変換できない場合はスキップ（競合とみなさない）

            results.append({
                '旧ラベル': old_label,
                '統一ラベル': new_label,
                '競合あり': 'はい' if strong_conflict else 'いいえ（片方が非パラメータ）',
                '競合件数': conflict_count,
                '競合証券番号（カンマ区切り）': ','.join([f"'{code}'" for code in conflict_codes])
            })
        else:
            results.append({
                '旧ラベル': old_label,
                '統一ラベル': new_label,
                '競合あり': 'いいえ',
                '競合件数': 0,
                '競合証券番号（カンマ区切り）': ''
            })
    else:
        results.append({
            '旧ラベル': old_label,
            '統一ラベル': new_label,
            '競合あり': 'データに存在しない',
            '競合件数': '-',
            '競合証券番号（カンマ区切り）': ''
        })

# 結果保存
conflict_df = pd.DataFrame(results)
conflict_df.to_csv('C:/bankdata/label_mapping_conflict_checked.csv', index=False, encoding='utf-8-sig')

print("✅ 修正済みの label_mapping_conflict_checked.csv を出力しました（証券番号の3桁区切り防止済み）。")


C:\Users\naoyasu.ikeda\AppData\Local\Temp\ipykernel_15488\1358844830.py:5: DtypeWarning: Columns (0,1,8,9,11,13,18,20,21,23,25,27,29,30,37,38,39,40,41,44,45,46,48,51,53,54,55,56,58,59,60,64,66,67,68,72,75,76,78,79,81,82,84,87,88,89,92,93,99,100,101,103,104,113,114,116,118,119,122,123,124,125,126,127,128,129,130,134,135,137,138,139,140,141,142,148,149,150,151,152,153,155,156,157,158,159,160,161,162,164,165,166,167,169,173,175,181,184,185,186,187,188,189,199,202,205,206,214,229,231,270,271,272,274,281,283,286,288,289,290,291,297,298,299,300,302,305,309,311,312,315,316,317,320,325,327,329,330,337,339,340,349,352,353,357,361,362,363,364,365,366,367,368,378,379,382,383,384,385,389,394,395,396,398,399,402,408,409,414,415,417,420,426,428,440,443,444,445,446,448,449,450,452,453,454,456,457,458,459,460,461,462,463,474,480,481,482,483,484,491,493,501,503,506,507,508,511,522,525,529,532,533,536,537,538,539,541,546,547,549,551,552,553,557,560,561,562,564,565,567,574,577,599,602,607,610,620,621,623

✅ 修正済みの label_mapping_conflict_checked.csv を出力しました（証券番号の3桁区切り防止済み）。


In [15]:
import pandas as pd
import numpy as np

# ファイルのパス
data_path = 'C:/bankdata/combined_dataset.csv'
map_path = 'C:/bankdata/conflict_map.csv'

# データ読み込み
df = pd.read_csv(data_path)
mapping_df = pd.read_csv(map_path)

# マッピング辞書
mapping_dict = dict(zip(mapping_df['旧ラベル'], mapping_df['統一ラベル']))

conflict_rows = []

for old_label, unified_label in mapping_dict.items():
    if old_label not in df.columns:
        continue

    if unified_label not in df.columns:
        continue

    for idx, row in df.iterrows():
        val_old = row[old_label]
        val_new = row[unified_label]

        # 両方が NaN ならスキップ
        if pd.isna(val_old) or pd.isna(val_new):
            continue

        # 両方とも数値に変換できるかをチェック
        try:
            num_old = float(val_old)
            num_new = float(val_new)
            # どちらも数値であり、両方に値がある → 競合と判断
            conflict_rows.append({
                '証券番号': row.get('証券番号', ''),
                '旧ラベル': old_label,
                '旧値': val_old,
                '統一ラベル': unified_label,
                '統一値': val_new,
                '行番号': idx
            })
        except ValueError:
            # 少なくとも片方が非数値（文字列） → 競合としない
            continue

# 結果を保存
conflict_df = pd.DataFrame(conflict_rows)
output_path = 'C:/bankdata/label_numeric_conflicts.csv'
conflict_df.to_csv(output_path, index=False, encoding='utf-8-sig')

if not conflict_df.empty:
    print(f"⚠️ 数値データの競合が {len(conflict_df)} 件見つかりました。詳細は以下のファイルを確認してください：\n{output_path}")
else:
    print("✅ 数値同士の競合は見つかりませんでした。")


C:\Users\naoyasu.ikeda\AppData\Local\Temp\ipykernel_9232\2222714290.py:9: DtypeWarning: Columns (0,1,2,8,9,11,13,18,20,21,25,27,29,30,37,38,39,40,43,44,45,46,48,51,53,54,55,56,57,63,67,70,71,75,78,81,82,84,85,87,90,91,92,95,96,97,103,105,110,112,113,114,116,117,118,120,122,123,131,132,134,136,137,140,141,142,143,144,145,146,147,148,149,153,159,160,161,162,164,165,166,167,168,169,170,171,172,175,176,177,182,183,184,185,187,189,190,191,192,193,194,195,196,198,204,205,206,207,208,209,210,211,212,214,215,218,219,220,223,224,225,227,228,229,231,232,233,237,238,244,246,249,251,257,258,261,265,268,269,271,273,275,278,279,281,284,285,286,287,291,292,301,302,312,315,316,320,324,337,338,342,344,387,403,404,405,407,419,421,422,423,424,430,431,432,433,435,440,447,452,461,463,465,466,473,475,476,485,488,490,494,499,500,501,502,504,505,506,507,519,520,523,524,525,527,531,536,537,538,540,541,544,550,554,555,559,561,562,563,566,567,568,571,573,574,575,576,577,581,582,584,585,586,587,588,590,612,614,61

✅ 数値同士の競合は見つかりませんでした。


In [18]:
import pandas as pd
import numpy as np

# --- ファイルパス設定 ---
data_path = 'C:/bankdata/combined_dataset.csv'
mapping_path = 'C:/bankdata/conflict_map.csv'
output_csv = 'C:/bankdata/base_dataset.csv'
conflict_log_path = 'C:/bankdata/conflict_log.txt'

# --- 読み込み ---
df = pd.read_csv(data_path)
mapping_df = pd.read_csv(mapping_path)

# --- マッピング辞書作成 ---
mapping_dict = dict(zip(mapping_df['旧ラベル'], mapping_df['統一ラベル']))

# --- 除外対象ラベル ---
excluded_columns = ['証券番号', '倒産フラグ', '連結フラグ']

# --- 統合用辞書と競合ログ ---
final_data = {}
conflict_log = []

# --- レコードごとの統合処理 ---
for _, row in df.iterrows():
    code = row['証券番号']
    if code not in final_data:
        final_data[code] = {'証券番号': code}

    for label, value in row.items():
        if label in excluded_columns:
            final_data[code][label] = value  # そのまま転記
            continue

        unified_label = mapping_dict.get(label, label)

        if unified_label in final_data[code]:
            if pd.notna(final_data[code][unified_label]) and pd.notna(value):
                # 値が両方あり → 数値として異なる場合のみ競合記録
                try:
                    v1 = float(final_data[code][unified_label])
                    v2 = float(value)
                    if not np.isclose(v1, v2, rtol=1e-5):
                        conflict_log.append(
                            f"証券番号 {code} のラベル '{unified_label}' に既存値 {v1} と新値 {v2} が競合"
                        )
                except:
                    # どちらかが文字列（数値化できない）場合はスキップ
                    pass
                continue  # 上書きせずスキップ

        if pd.notna(value):
            final_data[code][unified_label] = value

# --- DataFrame化と保存 ---
final_df = pd.DataFrame.from_dict(final_data, orient='index')
final_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"✅ 統合済みデータを {output_csv} に保存しました。")

# --- 競合ログ出力 ---
if conflict_log:
    with open(conflict_log_path, 'w', encoding='utf-8') as f:
        for entry in conflict_log:
            f.write(entry + '\n')
    print(f"⚠️ 競合ログを {conflict_log_path} に出力しました。")
else:
    print("✅ 競合は発生しませんでした。")


C:\Users\naoyasu.ikeda\AppData\Local\Temp\ipykernel_9232\1180268846.py:11: DtypeWarning: Columns (0,1,2,8,9,11,13,18,20,21,25,27,29,30,37,38,39,40,43,44,45,46,48,51,53,54,55,56,57,63,67,70,71,75,78,81,82,84,85,87,90,91,92,95,96,97,103,105,110,112,113,114,116,117,118,120,122,123,131,132,134,136,137,140,141,142,143,144,145,146,147,148,149,153,159,160,161,162,164,165,166,167,168,169,170,171,172,175,176,177,182,183,184,185,187,189,190,191,192,193,194,195,196,198,204,205,206,207,208,209,210,211,212,214,215,218,219,220,223,224,225,227,228,229,231,232,233,237,238,244,246,249,251,257,258,261,265,268,269,271,273,275,278,279,281,284,285,286,287,291,292,301,302,312,315,316,320,324,337,338,342,344,387,403,404,405,407,419,421,422,423,424,430,431,432,433,435,440,447,452,461,463,465,466,473,475,476,485,488,490,494,499,500,501,502,504,505,506,507,519,520,523,524,525,527,531,536,537,538,540,541,544,550,554,555,559,561,562,563,566,567,568,571,573,574,575,576,577,581,582,584,585,586,587,588,590,612,614,6

✅ 統合済みデータを C:/bankdata/base_dataset.csv に保存しました。
✅ 競合は発生しませんでした。


In [19]:
import pandas as pd
import os

# 元データ読み込み
df = pd.read_csv('C:/bankdata/base_dataset.csv', encoding='utf-8-sig')

# 保存先ディレクトリ
save_dir = 'C:/bankdata/resampled_datasets_combined'
os.makedirs(save_dir, exist_ok=True)

# 単体・連結に分離
standalone_df = df[df['連結フラグ'] == 0]
consolidated_df = df[df['連結フラグ'] == 1]

# リサンプリング関数（倒産証券番号も返す）
def resample_dataset(df, ratio=2, random_state=42):
    positive = df[df['倒産フラグ'] == 1]
    negative = df[df['倒産フラグ'] == 0]
    sampled_negative = negative.sample(n=min(len(positive)*ratio, len(negative)),
                                       random_state=random_state)
    return pd.concat([positive, sampled_negative]), len(positive), positive['証券番号'].tolist()

# 各比率で処理
ratios = [2, 5, 10, 20]
for ratio in ratios:
    sampled_s, count_s, codes_s = resample_dataset(standalone_df, ratio)
    sampled_c, count_c, codes_c = resample_dataset(consolidated_df, ratio)

    combined = pd.concat([sampled_s, sampled_c]).sample(frac=1, random_state=42).reset_index(drop=True)

    filename = f'combined_1_to_{ratio}.csv'
    combined.to_csv(os.path.join(save_dir, filename), index=False, encoding='utf-8-sig')

    print(f"✅ {filename} を保存しました")
    print(f"  単体倒産件数: {count_s} 件 / 非倒産: {len(sampled_s) - count_s}")
    print(f"    倒産証券番号: {', '.join(map(str, codes_s))}")
    print(f"  連結倒産件数: {count_c} 件 / 非倒産: {len(sampled_c) - count_c}")
    print(f"    倒産証券番号: {', '.join(map(str, codes_c))}")
    print("---")


C:\Users\naoyasu.ikeda\AppData\Local\Temp\ipykernel_9232\1576575647.py:5: DtypeWarning: Columns (0,1,3,4,6,8,10,11,12,14,16,17,18,23,24,25,26,29,30,31,33,35,37,38,39,40,43,44,45,48,49,51,52,54,56,57,58,59,61,62,63,64,65,66,67,68,69,71,72,73,74,75,76,77,78,79,80,81,82,83,84,86,90,91,92,93,94,95,96,97,98,100,101,103,104,105,107,108,109,110,111,113,114,115,117,118,119,122,125,127,128,129,135,142,144,145,146,148,149,150,151,152,153,154,160,161,173,174,176,179,189,190,194,196,216,232,233,234,237,238,239,240,241,249,251,252,253,258,259,260,261,264,273,275,277,278,285,287,288,297,300,301,305,309,310,311,312,313,314,315,316,327,328,331,332,333,335,339,344,345,346,348,349,352,356,358,359,363,365,366,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,385,387,390,391,394,396,398,399,400,402,403,404,405,406,407,408,409,420,422,423,426,431,434,435,437,438,439,440,441,442,443,444,445,446,449,452,455,660,661,665,672,678,679,683,684,701,703,705,711,715,716,721,725,730,731,735,736,737,738,740,

✅ combined_1_to_2.csv を保存しました
  単体倒産件数: 16 件 / 非倒産: 32
    倒産証券番号: 1817, 1839, 2205, 2228, 2473, 3767, 4357, 5917, 6660, 7633, 8489, 9204, 9710, E04668, E04706, E04725
  連結倒産件数: 70 件 / 非倒産: 140
    倒産証券番号: 1351, 1606, 1792, 1804, 1818, 1836, 1851, 1854, 1858, 1872, 1889, 2318, 2356, 2403, 3207, 3258, 3584, 3587, 3606, 4313, 4835, 5759, 6102, 6106, 6665, 6667, 6671, 6868, 6891, 7104, 7312, 7612, 7853, 7934, 8024, 8146, 8243, 8256, 8262, 8564, 8567, 8571, 8577, 8587, 8597, 8826, 8839, 8858, 8866, 8868, 8874, 8878, 8882, 8888, 8899, 8902, 8910, 8911, 8921, 8939, 8947, 9132, 9374, 9609, 9703, 9786, 9822, 9963, E03059, E04490
---
✅ combined_1_to_5.csv を保存しました
  単体倒産件数: 16 件 / 非倒産: 80
    倒産証券番号: 1817, 1839, 2205, 2228, 2473, 3767, 4357, 5917, 6660, 7633, 8489, 9204, 9710, E04668, E04706, E04725
  連結倒産件数: 70 件 / 非倒産: 350
    倒産証券番号: 1351, 1606, 1792, 1804, 1818, 1836, 1851, 1854, 1858, 1872, 1889, 2318, 2356, 2403, 3207, 3258, 3584, 3587, 3606, 4313, 4835, 5759, 6102, 6106, 6665, 6667, 6671,

In [4]:
import pandas as pd
import os

# 元データ読み込み
df = pd.read_csv('C:/bankdata/combined_dataset.csv', encoding='utf-8-sig')

# 保存先ディレクトリ
save_dir = 'C:/bankdata/resampled_datasets_combined'
os.makedirs(save_dir, exist_ok=True)

# 単体・連結に分離
standalone_df = df[df['連結フラグ'] == 0]
consolidated_df = df[df['連結フラグ'] == 1]

# リサンプリング関数（倒産証券番号も返す）
def resample_dataset(df, ratio=2, random_state=42):
    positive = df[df['倒産フラグ'] == 1]
    negative = df[df['倒産フラグ'] == 0]
    sampled_negative = negative.sample(n=min(len(positive)*ratio, len(negative)),
                                       random_state=random_state)
    return pd.concat([positive, sampled_negative]), len(positive), positive['証券番号'].tolist()

# 各比率で処理
ratios = [2, 5, 10, 20]
for ratio in ratios:
    sampled_s, count_s, codes_s = resample_dataset(standalone_df, ratio)
    sampled_c, count_c, codes_c = resample_dataset(consolidated_df, ratio)

    combined = pd.concat([sampled_s, sampled_c]).sample(frac=1, random_state=42).reset_index(drop=True)

    filename = f'combined_1_to_{ratio}.csv'
    combined.to_csv(os.path.join(save_dir, filename), index=False, encoding='utf-8-sig')

    print(f"✅ {filename} を保存しました")
    print(f"  単体倒産件数: {count_s} 件 / 非倒産: {len(sampled_s) - count_s}")
    print(f"    倒産証券番号: {', '.join(map(str, codes_s))}")
    print(f"  連結倒産件数: {count_c} 件 / 非倒産: {len(sampled_c) - count_c}")
    print(f"    倒産証券番号: {', '.join(map(str, codes_c))}")
    print("---")

C:\Users\naoyasu.ikeda\AppData\Local\Temp\ipykernel_36928\3269043083.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,64,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,108,109,112,113,114,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,171,172,173,175,176,178,179,180,181,184,185,186,187,188,189,196,199,202,205,206,210,211,212,214,218,219,227,229,231,248,253,270,271,272,274,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,295,296,297,298,299,300,301,302,305,307,309,310,311,312,314,315,316,317,319,320,321,322,325,327,329,330,337,339,340,343,349,352,353,356,357,360,361,362,363,364,365,

✅ combined_1_to_2.csv を保存しました
  単体倒産件数: 14 件 / 非倒産: 28
    倒産証券番号: 1817, 1839, 2205, 2473, 3767, 4357, 5917, 6660, 7633, 8489, 9204, 9710, E04668, E04706
  連結倒産件数: 70 件 / 非倒産: 140
    倒産証券番号: 1351, 1606, 1792, 1804, 1818, 1836, 1851, 1854, 1858, 1872, 1889, 2318, 2356, 2403, 3207, 3258, 3584, 3587, 3606, 4313, 4835, 5759, 6102, 6106, 6665, 6667, 6671, 6868, 6891, 7104, 7312, 7612, 7853, 7934, 8024, 8146, 8243, 8256, 8262, 8564, 8567, 8571, 8577, 8587, 8597, 8826, 8839, 8858, 8866, 8868, 8874, 8878, 8882, 8888, 8899, 8902, 8910, 8911, 8921, 8939, 8947, 9132, 9374, 9609, 9703, 9786, 9822, 9963, E03059, E04490
---
✅ combined_1_to_5.csv を保存しました
  単体倒産件数: 14 件 / 非倒産: 70
    倒産証券番号: 1817, 1839, 2205, 2473, 3767, 4357, 5917, 6660, 7633, 8489, 9204, 9710, E04668, E04706
  連結倒産件数: 70 件 / 非倒産: 350
    倒産証券番号: 1351, 1606, 1792, 1804, 1818, 1836, 1851, 1854, 1858, 1872, 1889, 2318, 2356, 2403, 3207, 3258, 3584, 3587, 3606, 4313, 4835, 5759, 6102, 6106, 6665, 6667, 6671, 6868, 6891, 7104, 7312, 761

In [4]:
final_df

,証券番号,連結貸借対照表_現金及び預金,連結貸借対照表_受取手形及び売掛金,連結貸借対照表_有価証券,連結貸借対照表_たな卸資産,連結貸借対照表_繰延税金資産,連結貸借対照表_その他の流動資産,連結貸借対照表_貸倒引当金,連結貸借対照表_流動資産合計,連結貸借対照表_減価償却累計額,...,貸借対照表_預り保証基金の増減（△は減少）,貸借対照表_事業税（外形標準課税）計上額,貸借対照表_開発費償却,貸借対照表_諸会費,貸借対照表_研修費,貸借対照表_交通費,貸借対照表_長期貸付金の業務委託費等相殺額,貸借対照表_関係会社社債,貸借対照表_新株式申込証拠金,貸借対照表_新株式申込証拠金の払込による収入
1351,1351,2670.0,8999.0,52.0,4935.0,264.0,42.0,-1431.0,19035.0,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,1606,17264.0,NaN,NaN,NaN,19.0,NaN,-886.0,29487.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1792,1792,NaN,NaN,NaN,NaN,-,NaN,-197.0,7947.0,145.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1804,1804,NaN,NaN,49.0,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1818,1818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-299.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E04813,E04813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9471.0,560.0,59.0,2223.0,NaN,NaN,NaN,NaN
E04843,E04843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E22464,E22464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1208.0,NaN,NaN,NaN
E23973,E23973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300000.0,NaN,NaN


In [6]:
final_df.columns

Index(['証券番号', '連結貸借対照表_現金及び預金', '連結貸借対照表_受取手形及び売掛金', '連結貸借対照表_有価証券',
       '連結貸借対照表_たな卸資産', '連結貸借対照表_繰延税金資産', '連結貸借対照表_その他の流動資産', '連結貸借対照表_貸倒引当金',
       '連結貸借対照表_流動資産合計', '連結貸借対照表_減価償却累計額',
       ...
       '貸借対照表_預り保証基金の増減（△は減少）', '貸借対照表_事業税（外形標準課税）計上額', '貸借対照表_開発費償却',
       '貸借対照表_諸会費', '貸借対照表_研修費', '貸借対照表_交通費', '貸借対照表_長期貸付金の業務委託費等相殺額',
       '貸借対照表_関係会社社債', '貸借対照表_新株式申込証拠金', '貸借対照表_新株式申込証拠金の払込による収入'],
      dtype='object', length=21657)

In [13]:
import os
import pandas as pd
import re

# --- 各カテゴリのフォルダを指定 ---
folders = {
    '倒産かつ連結': 'C:/bankdata/tousan/renketsu/',
    '倒産かつ単体': 'C:/bankdata/tousan/tantai/',
    '非倒産かつ連結': 'C:/bankdata/hitousan/renketsu/',
    '非倒産かつ単体': 'C:/bankdata/hitousan/tantai/'
}

# --- カテゴリごとのフラグ設定 ---
category_flags = {
    '倒産かつ連結': {'倒産フラグ': 1, '連結フラグ': 1},
    '倒産かつ単体': {'倒産フラグ': 1, '連結フラグ': 0},
    '非倒産かつ連結': {'倒産フラグ': 0, '連結フラグ': 1},
    '非倒産かつ単体': {'倒産フラグ': 0, '連結フラグ': 0}
}

# --- 全データを格納する辞書 ---
final_data = {}

# --- 各フォルダのCSVファイルを読み込み、横展開 ---
for category, folder_path in folders.items():
    for filename in os.listdir(folder_path):
        if not filename.endswith(".csv"):
            continue
        file_path = os.path.join(folder_path, filename)

        # 証券番号の抽出（先頭4桁 or E+5桁）
        match = re.match(r'(E?\d{4,5})', filename)
        if not match:
            continue
        company_code = match.group(1)

        # CSV読み込み
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except Exception as e:
            print(f"読み込み失敗: {filename}, エラー: {e}")
            continue

        if df.shape[1] < 4:
            continue  # 4列未満ならシート名列が存在しないのでスキップ

        # 1列目＝項目名, 4列目＝シート名 としてラベルを構築
        raw_labels = df.iloc[:, 0].astype(str).str.strip().str.replace(" ", "").str.replace("\n", "")
        sheet_labels = df.iloc[:, 3].astype(str).str.strip().str.replace(" ", "").str.replace("\n", "")
        labels = [f"{sheet}_{label}" for sheet, label in zip(sheet_labels, raw_labels)]

        # パラメータ列の候補（2列目 or 3列目）
        def numeric_ratio(col_index):
            return pd.to_numeric(df.iloc[:, col_index], errors='coerce').notna().mean()
        
        col2_ratio = numeric_ratio(1)
        col3_ratio = numeric_ratio(2) if df.shape[1] >= 3 else 0
        value_col = 1 if col2_ratio >= col3_ratio else 2

        values = df.iloc[:, value_col]

        # 辞書初期化
        if company_code not in final_data:
            final_data[company_code] = {'証券番号': company_code}

        # 横方向に追加
        for label, value in zip(labels, values):
            if label != "nan_nan":
                final_data[company_code][label] = value

        # 倒産・連結フラグ追加
        final_data[company_code]['倒産フラグ'] = category_flags[category]['倒産フラグ']
        final_data[company_code]['連結フラグ'] = category_flags[category]['連結フラグ']

# --- DataFrameに変換 ---
final_df = pd.DataFrame.from_dict(final_data, orient='index')

# --- 保存 ---
output_path = 'C:/bankdata/combined_dataset.csv'
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"結合されたデータセットを {output_path} に保存しました。")


結合されたデータセットを C:/bankdata/combined_dataset.csv に保存しました。


In [14]:
labels

['貸借対照表_表名称',
 '貸借対照表_nan',
 '貸借対照表_nan',
 '貸借対照表_nan',
 '貸借対照表_(千円)',
 '貸借対照表_2012/03/31現在',
 '貸借対照表_貸借対照表',
 '貸借対照表_資産の部',
 '貸借対照表_流動資産',
 '貸借対照表_現金及び預金',
 '貸借対照表_未収入金',
 '貸借対照表_商品',
 '貸借対照表_原材料及び貯蔵品',
 '貸借対照表_前払費用',
 '貸借対照表_その他',
 '貸借対照表_貸倒引当金',
 '貸借対照表_流動資産合計',
 '貸借対照表_固定資産',
 '貸借対照表_有形固定資産',
 '貸借対照表_建物',
 '貸借対照表_減価償却累計額及び減損損失累計額',
 '貸借対照表_建物（純額）',
 '貸借対照表_構築物',
 '貸借対照表_減価償却累計額及び減損損失累計額',
 '貸借対照表_構築物（純額）',
 '貸借対照表_機械及び装置',
 '貸借対照表_減価償却累計額及び減損損失累計額',
 '貸借対照表_機械及び装置（純額）',
 '貸借対照表_車両運搬具',
 '貸借対照表_減価償却累計額及び減損損失累計額',
 '貸借対照表_車両運搬具（純額）',
 '貸借対照表_工具、器具及び備品',
 '貸借対照表_減価償却累計額及び減損損失累計額',
 '貸借対照表_工具、器具及び備品（純額）',
 '貸借対照表_土地',
 '貸借対照表_コース勘定',
 '貸借対照表_有形固定資産合計',
 '貸借対照表_投資その他の資産',
 '貸借対照表_長期前払費用',
 '貸借対照表_保険積立金',
 '貸借対照表_その他',
 '貸借対照表_投資その他の資産合計',
 '貸借対照表_固定資産合計',
 '貸借対照表_資産合計',
 '貸借対照表_負債の部',
 '貸借対照表_流動負債',
 '貸借対照表_買掛金',
 '貸借対照表_未払金',
 '貸借対照表_未払費用',
 '貸借対照表_未払法人税等',
 '貸借対照表_預り金',
 '貸借対照表_前受収益',
 '貸借対照表_流動負債合計',
 '貸借対照表_固定負債',
 '貸借対照表_関係会社長期借入金',
 '貸借対照表_退職給付引当金',
 '貸借対照表_固定負債合計',
 '貸借対照表_負債合計',